In [ ]:
import pandas as pd
import numpy as np
import tushare as ts
import os 
from tqdm import tqdm
import glob
debug = False
from datetime import datetime 

In [ ]:
pro = ts.pro_api('20240522230128-d50d7a15-d7c5-47d4-bccc-25f6c754496a')
pro._DataApi__http_url = 'http://tsapi.majors.ltd:7000'

In [ ]:
trade_date = '20240530'
stocklist = pro.stock_basic(exchange='', list_status='L', 
                            fields='ts_code, symbol, name, area, industry, list_date')
stocklist.head()

In [ ]:
trade_cal = pro.trade_cal(exchange='', start_date='20220101', end_date='20240430')
trade_cal = trade_cal[trade_cal.is_open==1]
trade_cal.head()

In [ ]:
price_PD = []
for trade_date in trade_cal.cal_date:
    df = [] 
    for code in tqdm(stocklist.ts_code.head(30)):
        tmp = ts.pro_bar(ts_code=code, adj='qfq', start_date=trade_date, end_date=trade_date, api=pro)
        df.append(tmp)
    df = pd.concat(df)
    price_PD.append(df)
    
price_PD = pd.concat(price_PD).reset_index(drop=True)

price_PD = price_PD.drop_duplicates().sort_values(['ts_code','trade_date']).reset_index(drop=True)
price_PD.to_csv(f'price_PD.csv', index=False)  #前复权
print("Data saved.")


In [ ]:
price_PD = []
for trade_date in trade_cal.cal_date:
    df = [] 
    for code in tqdm(stocklist):
        tmp = ts.pro_bar(ts_code=code, adj=None, start_date=trade_date, end_date=trade_date, api=pro)
        df.append(tmp)
    df = pd.concat(df)
    price_PD.append(df)
    
price_PD = pd.concat(price_PD).reset_index(drop=True)

price_PD = price_PD.drop_duplicates().sort_values(['ts_code','trade_date']).reset_index(drop=True)
price_PD.to_csv('price_PD_unadj.csv', index=False)   #未复权/除权
print("Data saved.")